Import the libraries and the csv files.

In [1]:
import pandas as pd
import uuid
from io import StringIO
import re
from datetime import datetime

In [2]:
df_inventory = pd.read_csv("/content/drive/MyDrive/Analytics_Interview/inventory_data.csv")
df_majors = pd.read_csv("/content/drive/MyDrive/Analytics_Interview/majors_data.csv")
df_occupancy = pd.read_csv("/content/drive/MyDrive/Analytics_Interview/occupancy_data.csv")
df_persons = pd.read_csv("/content/drive/MyDrive/Analytics_Interview/persons_data.csv")

df_ inventory dataframe: Cleans and validates data integrity

Removes duplicates, separates rows with missing values, validates data types and removes inconsistencies.


In [3]:
# Check for duplicate entries in the bed inventory
assert not df_inventory[['buildingName', 'roomName', 'bedName']].duplicated().any()

# Identify rows with missing values and store them in a separate dataframe
df_missing_inventory_data = df_inventory[df_inventory.isnull().any(axis=1)]

# Remove rows with missing values from the main inventory dataframe
df_inventory = df_inventory.dropna()

# Verify that all missing values have been removed from the main inventory dataframe
assert df_inventory.isnull().sum().sum() == 0, "DataFrame still contains missing values"

# Check for consistent naming conventions
assert df_inventory['roomName'].str.startswith('Room-').all(), "Inconsistent room naming found"
assert df_inventory['bedName'].str.startswith('Bed-').all(), "Inconsistent bed naming found"

# Verify the uniqueness of bedId
assert df_inventory['bedId'].is_unique, "Duplicate bedId found"

# Check for valid room and bed numbers
df_inventory['roomNumber'] = df_inventory['roomName'].str.extract('(\d+)').astype(int)
df_inventory['bedNumber'] = df_inventory['bedName'].str.extract('(\d+)').astype(int)

# Display the first few rows of the cleaned and validated inventory dataframe
df_inventory.head()

,buildingName,roomName,bedName,bedId,roomNumber,bedNumber
0,Pine Hall,Room-1,Bed-1,2143605e-8854-414b-8ad1-0f0203a370d5,1,1
1,Pine Hall,Room-1,Bed-2,31a6601b-3b48-4e65-a064-75fc34cf80ea,1,2
2,Pine Hall,Room-2,Bed-1,67522423-d8c3-40d1-94dc-d000efb39064,2,1
3,Pine Hall,Room-2,Bed-2,31b16eda-2189-4f0b-89a8-f421ffb55b27,2,2
4,Pine Hall,Room-2,Bed-3,67466db1-4544-4812-a08d-147bf7eff4b1,2,3


df_majors dataframe: Cleans and validates data integrity

Handles duplicates, missing values, data types, and standardizes text fields. **bold text**

Assumption for keeping the last duplicate:
The decision to keep the last occurrence of each duplicate entry is based on the assumption that the most recent entry is likely to be the most up-to-date or correct version of the data.

In [4]:
# Storing the majors that are not recent
df_majors_dropped = df_majors[df_majors.duplicated(subset='name', keep='last')]

# Print the Dropped Majors DataFrame
df_majors_dropped.head()

,id,displayId,name,description
0,12c5d420-9bbc-455d-9794-7780bc5fea5b,d24fcc15-c1ac-46b1-aca8-dc998cf45e78,Computer Science,The study of computers and computational systems.
1,d3d06262-f539-4a05-9e71-6e6113e9529b,1e4c7f9f-40f5-4032-bd44-7036952e3105,Mechanical Engineering,The design and manufacture of mechanical systems.
2,19a8bee2-7566-411a-9919-61d44ccc81ef,c84821a7-9878-46e4-a666-34a538c10aa2,Electrical Engineering,The study of electricity and its applications.
3,b33ae65d-523e-48c6-b10e-59346e1c6523,d92604c6-1626-4794-9cce-cf0a7e136c19,Biology,The study of living organisms.
4,3c0ebd74-4105-4532-a321-39bc3d6ab3d8,7468a134-27ba-4c1e-8b24-8ff51cecfb5c,Chemistry,The study of substances and their properties.


In [5]:
# Check for missing values
assert df_majors.isnull().sum().sum() == 0, "DataFrame contains missing values"

# Identify duplicates based on major
duplicate_majors = df_majors[df_majors.duplicated(subset=['name'], keep=False)]

# Retain the last occurrence of each major
df_major_dropped_data = duplicate_majors.iloc[:-1, :]

# Keep only the last occurrence of each major
df_majors_cleaned = df_majors.drop_duplicates(subset=['name'], keep='last')

# Verify uniqueness of id and displayId
assert df_majors_cleaned['id'].is_unique, "Duplicate id found"
assert df_majors_cleaned['displayId'].is_unique, "Duplicate displayId found"

# Standardize text fields
df_majors_cleaned_copy = df_majors_cleaned.copy()
df_majors_cleaned_copy['name'] = df_majors_cleaned['name'].str.strip().str.title()
df_majors_cleaned_copy['description'] = df_majors_cleaned['description'].str.strip()
df_majors_cleaned = df_majors_cleaned_copy

# Check for minimum length in name and description
assert (df_majors_cleaned['name'].str.len() >= 3).all(), "Major names should be at least three characters long"

df_majors_cleaned.head()

,id,displayId,name,description
60,9a1dc165-a7e3-4d94-96b3-aed7943cd257,706c6c8e-1393-45ab-aaf3-da506ecf4e34,Music,"The study of musical theory, history, and perf..."
61,f5b20856-42b4-48bd-93d9-fbe2712c9e55,1abf440d-a728-4da3-8fc3-b2edf4779bb6,Theater,The study of drama and performance arts.
62,92ba9844-0c5e-4bfa-ad1c-ab41f8ac3d5e,309b4ad1-9540-4324-a5d8-21c507b5af91,Nursing,The study of patient care and medical assistance.
63,31a96803-e0b0-464c-b44c-14316de7195f,151efff6-2d2c-481c-8ef9-dbcb193eaa25,Health Science,The study of health and medical science.
64,ce83ec69-0901-4423-87f6-92734cc8feb2,730eb991-a367-4f80-8373-c00c0ee06e84,Environmental Science,The study of the environment and solutions to ...


df_occupancy dataframe: Cleans and validates data integrity

Handles missing values, duplicates, enforces uniqueness, and standardizes text fields.

In [6]:
# Assert no missing values
assert df_occupancy.isnull().sum().sum() == 0, "DataFrame contains missing values"

# Assert no duplicate rows
assert not df_occupancy.duplicated().any(), "DataFrame contains duplicate rows"

# Assert personId uniqueness
assert df_occupancy['personId'].nunique() == len(df_occupancy), "personId is not unique"

# Standardize text fields (buildingName, roomName)
df_occupancy['buildingName'] = df_occupancy['buildingName'].str.strip().str.title()
df_occupancy['roomName'] = df_occupancy['roomName'].str.strip().str.title()

# Ensure bedName follows expected format (Bed-1, Bed-2)
assert (df_occupancy['bedName'].str.startswith('Bed-') & df_occupancy['bedName'].str.contains('-')).all(), "Bed names must start with 'Bed-' and contain a hyphen"

df_occupancy.head()

,personId,buildingName,roomName,bedName
0,5594bbf8-f2fe-4a4f-b88f-54df252d0309,Pine Hall,Room-1,Bed-1
1,05ee7cdf-9306-492c-a2d1-3104928640ec,Pine Hall,Room-1,Bed-2
2,4b9c0758-de62-47d6-b78c-f29e754970af,Pine Hall,Room-2,Bed-1
3,045bdbc8-9945-4f75-bf1c-2a8079073d31,Pine Hall,Room-2,Bed-2
4,4b86c4b1-c1e1-4714-9b34-0153654200c1,Pine Hall,Room-2,Bed-3


df_persons dataframe: Cleans and validates data integrity

Handles missing values, duplicates, enforces uniqueness, and standardizes text fields.

Function to handle duplicate emails as the email field needs to be unique. Teh assumption is  that the latest data is the most  recent data hence retaining that data

In [7]:
def handle_duplicate_emails(df_persons):
    """
    This function checks for duplicate emails in a DataFrame of persons.

    @param df_persons: A DataFrame containing the persons' information.
    @return: A tuple containing the cleaned DataFrame and a DataFrame of duplicate emails.
    """
    # Standardize email column (optional lowercase conversion)
    df_persons['email'] = df_persons['email'].str.strip().str.lower()

    # Identify duplicate emails
    duplicates = df_persons[df_persons.duplicated(subset=['email'], keep=False)]

    # Save the dropped email entries
    df_email_dropped = duplicates.iloc[:-1, :]

    # Keep only the last occurrence of each duplicate email (for df_persons_cleaned)
    df_persons_cleaned = df_persons.drop_duplicates(subset=['email'], keep='last')

    return df_persons_cleaned, df_email_dropped

In [8]:
# Email validation function
def is_valid_email(email):
    """
    This function checks if an email address is valid.

    @param email: The email address to be validated.
    @return: True if the email is valid, False otherwise.
    """
    regex = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'

    return re.match(regex, email) is not None

# DOB validation function
def is_valid_dob(dob):
    """
    This function checks if a date of birth is valid.

    @param dob: The date of birth to be validated.
    @return: True if the date of birth is valid, False otherwise.
    """
    try:
        datetime.strptime(dob, '%Y-%m-%d')
        return True
    except ValueError:
        return False

# Address state validation function
def is_valid_state(address):
    """
    This function checks if an address' state is valid.

    @param address: The address to be validated.
    @return: True if the state is valid, False otherwise.
    """
    valid_states = {
        'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
        'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA',
        'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'
    }

    try:
        # Extract potential state abbreviation
        state = address.split(',')[-1].strip().split()[0]

        # Check for valid state
        return state.upper() in valid_states

    except IndexError:
        return False # Handle missing state information

In [9]:
def standardize_person_info(row):
    """
    This functions helps in stardizing the persons dataframe

    @param address: The address to be validated.
    @return: A DataFrame with standardized information.
    """

    # Standardize first name and last name
    first_name = row['firstName'].strip()
    last_name = row['lastName'].strip()

    # Standardize email
    email = row['email'].strip()

    # Standardize address
    address = row['address']
    if pd.notnull(address):
        parts = address.split(',')
        state = parts[-1].strip().upper()  # Uppercase state abbreviation
        city = ', '.join(parts[:-1]).strip()  # Rest of the address (preserve case)
        address = f"{city}, {state}"

    majors = row['majors']

    return pd.Series({
        'personId': row['personId'],
        'firstName': first_name,
        'lastName': last_name,
        'email': email,
        'dob': row['dob'],
        'address': address,
        'majors': row['majors']
    })



In [10]:
df_persons_cleaned, df_email_dropped = handle_duplicate_emails(df_persons)

# Find rows with missing values
missing_values = df_persons_cleaned.isnull().sum()
df_persons_missing = df_persons_cleaned[df_persons_cleaned.isnull().any(axis=1)]

# Drop the rows with missing values from the original dataframe
df_persons_cleaned = df_persons_cleaned.dropna()

# Check if all personids are unique
assert not df_persons_cleaned['personId'].duplicated(keep='first').any(), "DataFrame contains duplicate person IDs!"
df_persons_cleaned = df_persons_cleaned.apply(standardize_person_info, axis=1)

# Apply the validation functions
for index, row in df_persons_cleaned.iterrows():
    assert is_valid_email(row['email']), f"Invalid email: {row['email']} at index {index}"
    assert is_valid_dob(row['dob']), f"Invalid DOB: {row['dob']} at index {index}"
    assert is_valid_state(row['address']), f"Invalid state in address: {row['address']} at index {index}"

df_persons_cleaned.head()

,personId,firstName,lastName,email,dob,address,majors
4577,807db091-b8da-459a-8d1d-18e167350b32,Linda,Johnson,linda.johnson@example.com,1983-12-26,"646 Cedar St, Madison, TX","Communications, Computer Science, Computer Sci..."
4583,5b519619-c6e8-4887-8c6b-5950983a81f3,Sarah,Brown,sarah.brown@example.com,1993-06-28,"540 Spruce St, Salem, OH","Agriculture, Chemistry, Sociology, Music, Urba..."
4602,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,Jane,Doe,jane.doe@example.com,1962-03-10,"773 Maple St, Salem, FL","Philosophy, Chemistry, Library Science, Art"
4690,b9186768-cfb1-4187-9ab3-6fc1124f7e23,Michael,Davis,michael.davis@example.com,1995-11-26,"716 Oak St, Springfield, TX","Linguistics, Mathematics, Biology"
4714,6981a40f-c85a-4b86-aa41-2f57eb95dd95,David,Wilson,david.wilson@example.com,1990-09-06,"257 Birch St, Springfield, CA","Linguistics, Philosophy"


• split_address(df): parses address data into separate columns (address1, city, state, etc.) using a regular expression.

• combine_names(df): creates a new 'name' column by concatenating 'firstName' and 'lastName' columns.





In [11]:
def split_address(df):
    """
    This function helps in splitting the address column to multiple columns.

    @param df: The DataFrame containing the address column.
    @return: A DataFrame with the split address columns.
    """
    # Define a regex pattern for matching addresses
    address_pattern = r'^(?P<address1>.*?)(?:, (?P<address2>.*?))?, (?P<city>.*?), (?P<state>[A-Z]{2})(?: (?P<zipcode>\d{5}))?$'

    # Use str.extract to apply the regex pattern to all rows at once
    address_components = df['address'].str.extract(address_pattern)

    # Rename the columns to match the desired output
    address_components.columns = ['address1', 'address2', 'city', 'state', 'zipcode']

    # Replace NaN values with empty strings
    address_components = address_components.fillna('')

    # Combine the original dataframe with the new address components
    return pd.concat([df, address_components], axis=1)


def combine_names(df):
    """
    This function helps in combining the first and last names into a single 'name' column.

    @param df: The DataFrame containing the 'firstName' and 'lastName' columns.
    @return: A DataFrame with the 'name' column.
    """
    # Create a new column 'full_name' by concatenating 'firstName' and 'lastName'
    df['name'] = df['firstName'] + ' ' + df['lastName']

    return df



In [12]:
# Get the clean persons data
df_persons_cleaned = split_address(df_persons_cleaned)
df_persons_cleaned = combine_names(df_persons_cleaned)

# Drop the unnecessary columns
df_persons_cleaned.drop(columns=['address','firstName','lastName'], inplace=True)

# Rearrange columns and define column descriptions
df_persons_cleaned = df_persons_cleaned[['personId', 'name', 'email', 'dob', 'address1', 'address2', 'city', 'state', 'zipcode','majors']]

df_persons_cleaned.head()

,personId,name,email,dob,address1,address2,city,state,zipcode,majors
4577,807db091-b8da-459a-8d1d-18e167350b32,Linda Johnson,linda.johnson@example.com,1983-12-26,646 Cedar St,,Madison,TX,,"Communications, Computer Science, Computer Sci..."
4583,5b519619-c6e8-4887-8c6b-5950983a81f3,Sarah Brown,sarah.brown@example.com,1993-06-28,540 Spruce St,,Salem,OH,,"Agriculture, Chemistry, Sociology, Music, Urba..."
4602,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,Jane Doe,jane.doe@example.com,1962-03-10,773 Maple St,,Salem,FL,,"Philosophy, Chemistry, Library Science, Art"
4690,b9186768-cfb1-4187-9ab3-6fc1124f7e23,Michael Davis,michael.davis@example.com,1995-11-26,716 Oak St,,Springfield,TX,,"Linguistics, Mathematics, Biology"
4714,6981a40f-c85a-4b86-aa41-2f57eb95dd95,David Wilson,david.wilson@example.com,1990-09-06,257 Birch St,,Springfield,CA,,"Linguistics, Philosophy"


• Merged df_occupancy and df_inventory DataFrames on buildingName, roomName, and bedName (left join).

• Created df_occupancy_inventory with personId and bedId showing occupancy relationships.

In [13]:
# Filter occupancy data to keep only relevant persons
df_occupancy_filtered = df_occupancy[df_occupancy['personId'].isin(df_persons_cleaned['personId'])]

# Merge the occupancy and inventory dataframes
df_occupancy_inventory = pd.merge(df_occupancy_filtered, df_inventory, on=['buildingName', 'roomName', 'bedName'], how='left')

# Select relevant columns for the final result
df_occupancy_inventory = df_occupancy_inventory[['personId', 'bedId']]

# Check for missing bedId values
missing_values = df_occupancy_inventory['bedId'].isnull().sum()
print("Missing values" , missing_values)

# Assert that there are no missing bedId values
assert missing_values == 0, "There are missing bedId values in df_occupancy_inventory"

df_occupancy_inventory.head()

Missing values 0


,personId,bedId
0,b45d7f03-5b4e-400e-a4d3-30c2d6116caa,56a31cac-45da-4e72-ad96-27ea96d4c2d8
1,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,d5c4db6c-4203-4244-aeb6-df528b8f9603
2,b486891f-da16-452a-a27b-209a071c2f10,3310b826-1d0a-4cd9-a71b-786e71a767f5
3,0c95570f-4f20-4660-a6f4-5b32947aad70,97ecb11a-af8d-451e-ad4a-7057ca83e6fa
4,68319d83-f319-48cd-8d88-14e69ad08cfd,20e30621-5140-46c2-a9c5-57e96c06de3f


• Creates df_majors_persons linking students (df_persons_cleaned['personId']) to their majors (df_majors_cleaned).

• Uses a dictionary (major_name_to_id) to map majors to IDs and a function (get_major_ids) to process 'majors' column.

In [14]:
def create_majors_persons_df(df_persons_cleaned, df_majors_cleaned):
    """
    This function generates a new DataFrame df_majors_persons from df_persons_cleaned and df_majors_cleaned.
    Initially, it constructs a dictionary major_name_to_id mapping major names to their corresponding IDs extracted from df_majors_cleaned.
    Within the function, get_major_ids is defined to process each entry in the 'majors' column of df_persons_cleaned.

    @param df_persons_cleaned: The DataFrame containing the person information.
    @param df_majors_cleaned: The DataFrame containing the major information.
    @return: A DataFrame with 'personId' and 'majorIds' columns.
    """

    # Create a dictionary mapping major names to IDs
    major_name_to_id = dict(zip(df_majors_cleaned['name'], df_majors_cleaned['id']))

    def get_major_ids_from_names(majors_string):
        """
        This function processes a string of majors separated by commas.
        It splits the string into a list of majors, retrieves the IDs using the major_name_to_id dictionary,
        and filters out any empty IDs before joining them into a comma-separated
        string.

        @param majors_string: A string containing the majors separated by commas.
        @return: A comma-separated string of major IDs.
        """
        if pd.isna(majors_string):
            return ''

        majors = [m.strip() for m in majors_string.split(',')]
        major_ids = [major_name_to_id.get(major, '') for major in majors]
        return ','.join(filter(None, major_ids))

    # Create a new dataframe with personId and a new column for major IDs
    df_majors_persons = df_persons_cleaned[['personId']].copy()
    df_majors_persons['majorIds'] = df_persons_cleaned['majors'].apply(get_major_ids_from_names)

    return df_majors_persons

In [15]:
df_majors_persons = create_majors_persons_df(df_persons_cleaned, df_majors_cleaned)

df_majors_persons.head()

,personId,majorIds
4577,807db091-b8da-459a-8d1d-18e167350b32,"c8e9582b-d7bd-48cb-9cab-9f953712e952,91d3ffda-..."
4583,5b519619-c6e8-4887-8c6b-5950983a81f3,"15603f87-6ae3-403c-aad6-3cd82bdddc1e,32a60bdd-..."
4602,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,"a680ecae-78a0-4d78-88cd-4f0163e7d4da,32a60bdd-..."
4690,b9186768-cfb1-4187-9ab3-6fc1124f7e23,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,4ad06de7-..."
4714,6981a40f-c85a-4b86-aa41-2f57eb95dd95,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,a680ecae-..."


Combining the dataframes on personId to get all the columns required in the final output.

In [16]:
# Merge the occupancy and persons dataframe
df_combined = pd.merge(df_persons_cleaned, df_occupancy_inventory, on='personId', how='left')
df_combined = pd.merge(df_combined, df_majors_persons, on='personId', how='left')

# Arrange columns in the specified order
column_order = [
    'personId', 'name', 'email', 'dob', 'address1', 'address2', 'city', 'state', 'zipcode', 'majorIds', 'bedId'
]
df_combined = df_combined[column_order]

df_combined.head()

,personId,name,email,dob,address1,address2,city,state,zipcode,majorIds,bedId
0,807db091-b8da-459a-8d1d-18e167350b32,Linda Johnson,linda.johnson@example.com,1983-12-26,646 Cedar St,,Madison,TX,,"c8e9582b-d7bd-48cb-9cab-9f953712e952,91d3ffda-...",NaN
1,5b519619-c6e8-4887-8c6b-5950983a81f3,Sarah Brown,sarah.brown@example.com,1993-06-28,540 Spruce St,,Salem,OH,,"15603f87-6ae3-403c-aad6-3cd82bdddc1e,32a60bdd-...",NaN
2,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,Jane Doe,jane.doe@example.com,1962-03-10,773 Maple St,,Salem,FL,,"a680ecae-78a0-4d78-88cd-4f0163e7d4da,32a60bdd-...",d5c4db6c-4203-4244-aeb6-df528b8f9603
3,b9186768-cfb1-4187-9ab3-6fc1124f7e23,Michael Davis,michael.davis@example.com,1995-11-26,716 Oak St,,Springfield,TX,,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,4ad06de7-...",NaN
4,6981a40f-c85a-4b86-aa41-2f57eb95dd95,David Wilson,david.wilson@example.com,1990-09-06,257 Birch St,,Springfield,CA,,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,a680ecae-...",NaN


Refining the dataframe as their that  NULL bedId values and that field cannot be null.

In [17]:
# Check for missing bedId and majorID values
missing_value_major = df_combined['majorIds'].isnull().sum()
print(f"Missing majorIds: {missing_value_major}")

missing_value_bedID = df_combined['bedId'].isnull().sum()
print(f"Missing bedId: {missing_value_bedID}")

Missing majorIds: 0
Missing bedId: 43


In [18]:
# Filter rows where bedId is null
df_missing_bedId = df_combined[df_combined['bedId'].isnull()]

# Create a final dataframe
df_final = df_combined.dropna(subset=['bedId'])

df_final.head()

,personId,name,email,dob,address1,address2,city,state,zipcode,majorIds,bedId
2,bef9c3d3-efbd-4d40-aa48-1d43be75a89c,Jane Doe,jane.doe@example.com,1962-03-10,773 Maple St,,Salem,FL,,"a680ecae-78a0-4d78-88cd-4f0163e7d4da,32a60bdd-...",d5c4db6c-4203-4244-aeb6-df528b8f9603
5,f9f7459a-abe8-4517-b405-322900931df6,Alice Williams,alice.williams@example.com,1979-09-16,262 Spruce St,,Greenville,PA,,"fcda77f1-d71f-4cbe-bb5c-fb12877aec40,551cfc0c-...",3e942384-26f7-4b2a-8bcd-575a2fb55ee6
6,5d20c644-45ae-4ba0-a085-f921df9a6b77,John Williams,john.williams@example.com,1973-11-03,345 Elm St,,Riverside,NY,,"bd5df5b0-28d6-45d5-8637-31903ae85330,9a1dc165-...",da95ed54-cfa4-4d43-a13f-b0e81a63727f
7,646df5fa-d628-42a6-b550-b91c2071fdcf,Michael Moore,michael.moore@example.com,1998-09-23,875 Spruce St,,Fairview,PA,,"c6f0e3ea-fc4a-4768-a155-f0bf0b943717,91d3ffda-...",e3f9ab23-56b0-4530-acbe-c57f532d8c41
8,adc1531c-b933-4ecf-9201-ade32f20150f,Jane Smith,jane.smith@example.com,1976-08-17,155 Cedar St,,Salem,OH,,"551cfc0c-5de9-4672-90a2-b0f5b644ed8c,9630ccdb-...",dea5e5e5-948a-42c9-9edb-8fc3e37895e3


In [19]:

df_missing_bedId.head()


,personId,name,email,dob,address1,address2,city,state,zipcode,majorIds,bedId
0,807db091-b8da-459a-8d1d-18e167350b32,Linda Johnson,linda.johnson@example.com,1983-12-26,646 Cedar St,,Madison,TX,,"c8e9582b-d7bd-48cb-9cab-9f953712e952,91d3ffda-...",NaN
1,5b519619-c6e8-4887-8c6b-5950983a81f3,Sarah Brown,sarah.brown@example.com,1993-06-28,540 Spruce St,,Salem,OH,,"15603f87-6ae3-403c-aad6-3cd82bdddc1e,32a60bdd-...",NaN
3,b9186768-cfb1-4187-9ab3-6fc1124f7e23,Michael Davis,michael.davis@example.com,1995-11-26,716 Oak St,,Springfield,TX,,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,4ad06de7-...",NaN
4,6981a40f-c85a-4b86-aa41-2f57eb95dd95,David Wilson,david.wilson@example.com,1990-09-06,257 Birch St,,Springfield,CA,,"bc618e46-adeb-4bfa-a8dc-9e7a3bbad058,a680ecae-...",NaN
11,67f0a09a-1fd7-455c-8f56-64a2f7423336,Bob Smith,bob.smith@example.com,1961-01-10,394 Spruce St,,Springfield,FL,,"1f2a78f5-deae-438d-be76-6d733a7dac87,52e4d490-...",NaN


In [20]:
# Saving final dataframe to disk
df_final.to_csv('df_final.csv', index=False)

In [21]:
# Saving all the dropped data to disk
df_email_dropped.to_csv('df_email_dropped.csv', index=False)
df_persons_missing.to_csv('df_persons_missing.csv', index=False)
df_majors_dropped.to_csv('df_majors_dropped.csv', index=False)
df_missing_inventory_data.to_csv('df_missing_inventory_data.csv', index=False)
df_missing_bedId.to_csv('df_missing_bedId.csv', index=False)
